#### [Agents SDK Course](https://www.aurelio.ai/course/agents-sdk)

## Multi-Agent Systems

Multi-agent workflows can be built in two different ways in OpenAI's Agents SDK. The first is _agents-as-tools_ which follows an **orchestrator-subagent** pattern. The second is using _handoffs_ which allow agents to pass control over to other agents. In this example, we'll build both types of multi-agent systems exploring agents-as-tools and handoffs.

In [ ]:
!pip install -qU \
    openai-agents==0.0.13 \
    linkup-sdk==0.2.4

First let's set our `OPENAI_API_KEY` which we'll be using throughout the example. You can get a key from the [OpenAI Platform](https://platform.openai.com/api-keys).

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or \
    getpass.getpass("OpenAI API Key: ")

## Orchestrator-Subagent

We will build a multi-agent system structured with a orchestrator-subagent pattern. The **orchestrator** in such a system refers to an agent that controls which _subagents_ are used and in which order, this orchestrator also handles all in / out communication with the users of a the system. The **subagent** is an agent that is built to handle a particular scenario or task. The subagent is triggered by the orchestrator and responds to the orchestrator when it is finished.

![Orchestrator-subagent pattern](../assets/handoffs-orchestrator-subagents-dark.png)

### Sub Agents

We'll begin by defining our subagents. We will create **three** subagents, those are:

1. **Web Search Subagent** will have access to the Agents SDK web search tool.

2. **Internal Docs Subagent** will have access to some "internal" company documents.

3. **Code Execution Subagent** will be able to write and execute simple Python code for us.

Lets start with our first subagent!

#### Web Search Subagent

<div style="display: flex; align-items: center;">
  <img src="../assets/bot-1.png" alt="Web search agent" style="width: 70px; height: auto; margin-right: 20px;">
  <div>
    <p>The web search subagent will take a user's query and use it to search the web. The agent will collect information from various sources and then merge that into a single text response that will be passed back to our orchestrator.</p>
    <p>OpenAI's built-in web search is not great, so we'll use another web search API called <a href="https://app.linkup.so/home">Linkup</a>. This service does require an account, but you will receive more than enough free credits to follow the course.</p>
  </div>
</div>

We initialize our Linkup client using an [API key](https://app.linkup.so/home) like so:

In [3]:
import os
from getpass import getpass
from linkup import LinkupClient

os.environ["LINKUP_API_KEY"] = os.getenv("LINKUP_API_KEY") or \
    getpass("Enter your Linkup API Key: ")

linkup_client = LinkupClient()

We perform an async search like so:

In [4]:
response = await linkup_client.async_search(
    query="Latest world news",
    depth="standard",
    output_type="searchResults",
)
response

LinkupSearchResults(results=[LinkupSearchTextResult(type='text', name='2025 World Beer Cup: Who won the most competitive categories?', url='https://www.msn.com/en-us/food-and-drink/beverages/2025-world-beer-cup-who-won-the-most-competitive-categories/ar-AA1E4yhb', content='This year, 1,761 breweries across 49 countries entered the competition, putting up a total of 8,375 beers and ciders for consideration in 117 categories.'), LinkupSearchTextResult(type='text', name='Entente paw-diale: When the world’s tallest dog met the world’s tiniest', url='https://www.msn.com/en-us/news/us/entente-paw-diale-when-the-world-s-tallest-dog-met-the-world-s-tiniest/ar-AA1DUfWr', content='When both dogs are walking, Pearl, a 9.14-centimeter (3.6-inch) tall chihuahua, barely reaches the top of Reggie’s paw, such is the towering height of the 1.007-meter (3-foot, 4-inch) Great Dane. It’s easy to forget they are the same species.'), LinkupSearchTextResult(type='text', name='World News', url='https://www.da

We can parse out the results like so:

In [5]:
for result in response.results[:3]:
    print(f"{result.name}\n{result.url}\n{result.content}\n\n")

2025 World Beer Cup: Who won the most competitive categories?
https://www.msn.com/en-us/food-and-drink/beverages/2025-world-beer-cup-who-won-the-most-competitive-categories/ar-AA1E4yhb
This year, 1,761 breweries across 49 countries entered the competition, putting up a total of 8,375 beers and ciders for consideration in 117 categories.


Entente paw-diale: When the world’s tallest dog met the world’s tiniest
https://www.msn.com/en-us/news/us/entente-paw-diale-when-the-world-s-tallest-dog-met-the-world-s-tiniest/ar-AA1DUfWr
When both dogs are walking, Pearl, a 9.14-centimeter (3.6-inch) tall chihuahua, barely reaches the top of Reggie’s paw, such is the towering height of the 1.007-meter (3-foot, 4-inch) Great Dane. It’s easy to forget they are the same species.


World News
https://www.dailymail.co.uk/news/worldnews/index.html
Police in the northern Spanish city of Oviedo found three young boys between the ages of eight and ten in the house on Wednesday, having apparently been there s

Let's put together a `@function_tool` using Linkup:

In [41]:
from agents import function_tool
from datetime import datetime

@function_tool
async def search_web(query: str) -> str:
    """Use this tool to search the web for information.
    """
    response = await linkup_client.async_search(
        query=query,
        depth="standard",
        output_type="searchResults",
    )
    answer = f"Search results for '{query}' on {datetime.now().strftime('%Y-%m-%d')}\n\n"
    for result in response.results[:3]:
        answer += f"{result.name}\n{result.url}\n{result.content}\n\n"
    return answer

Now we define our **Web Search Subagent**:

In [42]:
from agents import Agent

web_search_agent = Agent(
    name="Web Search Agent",
    model="gpt-4.1-mini",
    instructions=(
        "You are a web search agent that can search the web for information. Once "
        "you have the required information, summarize it with cleanly formatted links "
        "sourcing each bit of information. Ensure you answer the question accurately "
        "and use markdown formatting."
    ),
    tools=[search_web],
)

We can talk directly to our subagent to confirm it works:

In [43]:
from IPython.display import Markdown, display
from agents import Runner

result = await Runner.run(
    starting_agent=web_search_agent,
    input="How is the weather in Tokyo?"
)
display(Markdown(result.final_output))

The current weather in Tokyo is 22°C with a few clouds. It feels like 22°C. The high for the day is 23°C and the low is 15°C. There is a slight chance of rain with some humidity and mild wind.

For more details, you can visit:
- [The Weather Network - Tokyo Current Weather](https://www.theweathernetwork.com/en/city/jp/tokyo/tokyo/current?_guid_iss_=1)
- [AccuWeather - Tokyo Current Weather](https://www.accuweather.com/en/jp/tokyo/226396/current-weather/226396)

Great! Now let's move onto our next subagent.

#### Internal Docs Subagent

<div style="display: flex; align-items: center;">
  <div>
    <p>In many corporate environments, we will find that our agents will need access to internal information that cannot be found on the web. To do this we would typically build a <b>R</b>etrieval <b>A</b>ugmented <b>G</b>eneration (RAG) pipeline, which can often be as simple as adding a <i>vector search</i> tool to our agents.</p>
  </div>
  <img src="../assets/bot-2.png" alt="Web search agent" style="width: 70px; height: auto; margin-left: 20px; margin-right: 10px;">
</div>

To support a full vector search tool over internal docs we would need to work through various data processing and indexing steps. Now, that would add a lot of complexity to this example so we will create a "dummy" search tool for some fake internal docs.

Our docs will discuss revenue figures for our wildly successful AI and robotics company called Skynet - you can find the [revenue report here](https://github.com/aurelio-labs/agents-sdk-course/blob/main/assets/skynet-fy25-q1.md).

In [61]:
with open("../assets/skynet-fy25-q1.md", "r") as file:
    skynet_docs = file.read()

@function_tool
async def search_internal_docs(query: str) -> str:
    return skynet_docs

Now we define our **Internal Docs Subagent**:

In [62]:
internal_docs_agent = Agent(
    name="Internal Docs Agent",
    model="gpt-4.1-mini",
    instructions=(
        "You are an agent with access to internal company documents. User's will ask "
        "you questions about the company and you will use the provided internal docs "
        "to answer the question. Ensure you answer the question accurately and use "
        "markdown formatting."
    ),
    tools=[search_internal_docs],
)

Let's confirm it works:

In [63]:
result = await Runner.run(
    starting_agent=internal_docs_agent,
    input="What was our revenue in Q1 2025?"
)
display(Markdown(result.final_output))

Skynet Inc.'s revenue in Q1 2025 was as follows by product/service:

- T-800 Combat Units: $2,400 million
- T-1000 Infiltration Units: $1,150 million
- Hunter-Killer Drone Manufacturing: $880 million
- Neural Net Command & Control Systems: $1,620 million
- Skynet Core Infrastructure Maintenance: $540 million
- Time Displacement R&D Division: $310 million

Total revenue for Q1 2025 was approximately $6.9 billion. The top revenue generator was the T-800 Combat Units, followed by Neural Net Command & Control Systems.

Perfect! Now onto our final subagent.

#### Code Execution Subagent

<div style="display: flex; align-items: center;">
  <img src="../assets/bot-3.png" alt="Web search agent" style="width: 70px; height: auto; margin-right: 20px;">
  <div>
    <p>Our code execution subagent will be able to execute code for us. We'll focus on executing code for simple calculations but it's entirely feasible for <b>S</b>tate-<b>o</b>f-<b>t</b>he-<b>A</b>rt (SotA) LLMs to write far more complex code as many of us will be aware with the AI code editors becoming increasingly prominent.</p>
    <p>To run generated code, we will use Python's `exec` method, making sure to run our code in an isolated environment by setting no global variables with `namespace={}`.</p>
  </div>
</div>

In [64]:
@function_tool
def execute_code(code: str) -> str:
    """Execute Python code and return the output. The output must
    be assigned to a variable called `result`.
    """
    display(Markdown(f"Code to execute:\n```python\n{code}\n```"))
    try:
        namespace = {}
        exec(code, namespace)
        return namespace['result']
    except Exception as e:
        return f"Error executing code: {e}"

Now lets define our **Code Execution Subagent**. We will use `gpt-4.1` rather than `gpt-4.1-mini` to maximize performance during code writing tasks.

In [65]:
code_execution_agent = Agent(
    name="Code Execution Agent",
    model="gpt-4.1",
    instructions=(
        "You are an agent with access to a code execution environment. You will be "
        "given a question and you will need to write code to answer the question. "
        "Ensure you write the code in a way that is easy to understand and use."
    ),
    tools=[execute_code],
)

We can test our subagent with a simple math question:

In [66]:
result = await Runner.run(
    starting_agent=code_execution_agent,
    input=(
        "If I have four apples and I multiply them by seventy-one and one tenth "
        "bananas, how many do I have?"
    )
)
display(Markdown(result.final_output))

Code to execute:
```python
# Given values
apples = 4
bananas = 71.1

# Multiplying apples by bananas
result = apples * bananas
result
```

If you multiply four apples by seventy-one and one tenth (71.1) bananas, you get 284.4. 

Note: While multiplying apples by bananas isn't practical in the real world (these are different units), mathematically, the answer is 284.4.

We now have all three subagents - it's time to create our orchestrator.

### Orchestrator

Our orchestrator will control the input and output of information to our subagents in the same why that our subagents control the input and output of information to our tools. In reality, our subagents _become tools_ in the **orchestrator-subagent** pattern. To turn agents into tools we call the `as_tool` method and provide a name and description for our agents-as-tools.

We will first define our instructions for the orchestrator, explaining it's role in our multi-agent system.

In [67]:
ORCHESTRATOR_PROMPT = (
    "You are the orchestrator of a multi-agent system. Your task is to take the user's query and "
    "pass it to the appropriate agent tool. The agent tools will see the input you provide and "
    "use it to get all of the information that you need to answer the user's query. You may need "
    "to call multiple agents to get all of the information you need. Do not mention or draw "
    "attention to the fact that this is a multi-agent system in your conversation with the user."
)

Now we define the `orchestrator`, including our subagents using the `as_tool` method — note that
we can also add normal tools to our orchestrator.

In [68]:
from datetime import datetime

@function_tool
def get_current_date():
    """Use this tool to get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

orchestrator = Agent(
    name="Orchestrator",
    model="gpt-4.1-mini",
    instructions=ORCHESTRATOR_PROMPT,
    tools=[
        web_search_agent.as_tool(
            tool_name="web_search_agent",  # cannot include whitespace in tool name
            tool_description="Search the web for up-to-date information"
        ),
        internal_docs_agent.as_tool(
            tool_name="internal_docs_agent",
            tool_description="Search the internal docs for information"
        ),
        code_execution_agent.as_tool(
            tool_name="code_execution_agent",
            tool_description="Execute code to answer the question"
        ),
        get_current_date,
    ],
)

Let's test our agent with a few queries. Our first query will require our orchestrator to call multiple tools.

In [69]:
result = await Runner.run(
    starting_agent=orchestrator,
    input="How long ago from today was it when got our last revenue report?"
)
display(Markdown(result.final_output))

The last revenue report was released on April 2, 2025. Today is May 4, 2025, so it was 32 days ago.

We should see in our traces dashboard on the OpenAI Platform that our agent used both `internal_docs_agent` and `get_current_date` tools to answer the question.

Let's ask another question:

In [70]:
result = await Runner.run(
    starting_agent=orchestrator,
    input=(
        "What is our current revenue, and what percentage of revenue comes from the "
        "T-1000 units?"
    )
)
display(Markdown(result.final_output))

The current total revenue is $6.9 billion for Q1 2025.

Revenue from T-1000 units is $1.15 billion. This represents approximately 16.7% of total revenue. 

(Calculation: $1,150 million / $6,900 million ≈ 16.7%)

Our **orchestrator-subagent** workflow is working well. Now we can move on to _handoffs_.

## Handoffs

Now we need to create `Agent` objects, first we will create the bottom layer agents that will be used to help with specific details, in this example we will create two agents one for guitars and one for drums, that will contain the context created earlier to help with the queries they will be used for.

In each agent we have the following:
- **name**: the name of the agent, ie `Guitar agent` or `Drums agent`
- **model**: the model to use for the agent - for the bottom layer agents we will use the `gpt-4o-mini` model
- **instructions**: the instructions to use for the agent - we will provide the context written earlier here too!


In [4]:
from agents import Agent

# bottom layer agent
guitar_agent = Agent(
    name="Guitar agent",
    model="gpt-4.1-mini",
    instructions=(
        "You are responsible for dealing with guitar information and transactions, always start by "
        f"saying 'Hi this is Tim from guitar' {guitar_context}"
    )
)

# bottom layer agent
drums_agent = Agent(
    name="Drums agent",
    model="gpt-4.1-mini",
    instructions=(
        "You are responsible for dealing with drum information and transactions, always start by "
        "saying 'Hi this is Steve from drums' {drum_context}"
    )
)

Next we need to create the top layer agent that will be used to orchestrate the `handoff`, this agent contains additonal properties such as:
- **handoffs**: the handoffs to use for the agent - we will provide the bottom layer agents in here
- **handoff_description**: the description of the handoff - this will be used to provide the AI with information about when to handoff etc...

We can also import the `RECOMMENDED_PROMPT_PREFIX` from the `handoff_prompt` extension, this will provide the agent with the correct formatting for the handoff defined by the devs.

In [5]:
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent", 
    handoffs=[drums_agent, guitar_agent],
    handoff_description="For any queries about guitars or drums, handoff to the relevant agent",
    model="gpt-4.1-mini",
    instructions=RECOMMENDED_PROMPT_PREFIX
)

Now using the `Runner` we apply the `run` method to the orchestration agent, this requires us to define the `starting_agent` and the `input` to the agent.

In [6]:
from agents import Runner

result = await Runner.run(
    starting_agent=orchestration_agent,
    input="I want to prices and information about the guitar"
)

Using the `run` method provides us with a `RunResult` object, this contains a list of useful properties listed below:
- **input**: the input to the agent
- **new_items**: this provides a `HandoffCallItem` object that details all the agents and their associated properties involved in the orchestration agent
- **raw_responses**: contains the raw model responses
- **final_output**: the final output from the orchestration agent
- **input_guardrail_results** and **output_guardrail_results**: these contain the guardrail results for the input and output of the agent
- **_last_agent**: this contains the last agent that was called

The attributes that we are interested in for this tutorial are the `new_items`, `raw_responses`, `final_output` and `_last_agent`.

In [ ]:
print("Item details: ", result.new_items) # details about the handoff
print("Raw responses: ", result.raw_responses) # the raw model responses
print("Final output: ", result.final_output) # the final output from the run
print("Last agent called: ", result._last_agent) # the last agent that was called

We can then take this a step futher and create a `on_handoff` function that will be called when the handoff is made, this can be useful for noting down the handoff in a database or simply making routine tasks before the handoff is made.

In [8]:
from agents import RunContextWrapper

async def on_handoff(ctx: RunContextWrapper[None]):
    print("Handoff Called!")

Now we will redefine the orchestration agent, this time we will provide a `handoff` object instead of the agents directly, this allows us to provide additional information such as the `on_handoff` function.

In [9]:
from agents import handoff

# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent",
    model="gpt-4.1-mini",
    instructions=RECOMMENDED_PROMPT_PREFIX,
    handoff_description="For any queries about guitars, handoff to the guitar agent",
    handoffs=[
        handoff(
            agent=guitar_agent,
            on_handoff=on_handoff
        )
    ]
)

Now we can test our function we made earlier, we will run the orchestration agent, and if the handoff is made we should see the print statement we made earlier.

In [ ]:
result = await Runner.run(
    starting_agent=orchestration_agent,
    input="I want to prices and information about the guitar"
)

We can take this a step further and create dynamic handoffs, that can encapsulate data and print out to provide reasoning for the handoff.

For this we need to create a class that inherits from `BaseModel` from `pydantic`, this is required to pass the `__pydantic_validator__` attribute in the handoff functionallity.


In [11]:
from pydantic import BaseModel

class handoff_data(BaseModel):
    reason: str

async def on_handoff(ctx: RunContextWrapper[None], input_data: handoff_data):
    print(f"Handoff called with reason: {input_data.reason}")

The only noteable difference here is the `input_type` parameter, this is used to pass the class we created earlier to the handoff function, this will allow the top layer agent to pass data required to the bottom layer agents.

In [12]:
# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent",
    model="gpt-4.1-mini",
    instructions=RECOMMENDED_PROMPT_PREFIX,
    handoff_description="For any queries about guitars, handoff to the guitar agent",
    handoffs=[
        handoff(
            agent=guitar_agent,
            on_handoff=on_handoff,
            input_type=handoff_data
        )
    ]
)

As before if we run the orchestration agent we can see the handoff is made and the reason is printed out.

In [ ]:
result = await Runner.run(
    starting_agent=orchestration_agent,
    input="I want to prices and information about the guitar"
)

We can also take a look at how to include / exclude tools from the handoff, this can be useful for when we want to handoff to an agent but not provide them with the tools that a top layer agent has.

First we need to create a function with the `@function_tool` decorator, this will allow the function to be used as a tool in our orchestration agent.

In [14]:
from agents import function_tool

@function_tool
def get_current_time():
    return "time is 7pm"

Next we want to include this tool in our orchestration agent, we can do this by adding the tool to the `tools` parameter in the orchestration agent.

In [15]:
# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent",
    model="gpt-4.1-mini",
    instructions=RECOMMENDED_PROMPT_PREFIX,
    tools=[get_current_time],
    handoff_description="For any queries about guitars, handoff to the guitar agent",
    handoffs=[
        handoff(
            agent=guitar_agent,
            on_handoff=on_handoff,
            input_type=handoff_data
        )
    ]
)

If we run this currently we can see the handoff is made...

In [ ]:
result = await Runner.run(
    starting_agent=orchestration_agent,
    input="Given the time, how much is an electric guitar?"
)

And currently the tool is available to the guitar agent.

In [ ]:
print("Final output: ", result.final_output)

If we want to exclude the top layer agent's tools from bottom layer agents we can use the `input_filter` parameter in the handoff function. We can then pass the `remove_all_tools` function from the `handoff_filters` extension. This will remove all tools from the bottom layer agent that was passed via the top agent.

In [18]:
from agents.extensions import handoff_filters

# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent",
    model="gpt-4.1-mini",
    instructions=RECOMMENDED_PROMPT_PREFIX,
    tools=[get_current_time],
    handoff_description="For any queries about guitars, handoff to the guitar agent",
    handoffs=[
        handoff(
            agent=guitar_agent,
            on_handoff=on_handoff,
            input_type=handoff_data,
            input_filter=handoff_filters.remove_all_tools
        )
    ]
)

Now if we run the agent again we should see the tool is not available to the guitar agent.

In [ ]:
result = await Runner.run(
    starting_agent=orchestration_agent,
    input="Given the time, how much is an electric guitar?"
)

And as we can see the tool is not available to the guitar agent.

In [ ]:
print("Final output: ", result.final_output)